# Boids Simulation 2.0:
- The first simulation I wrote worked however it suffered from computational complexity O(n^2) for the distance calculations
- This code uses a different programming paradigm so that it can take advantage of certain computational advantages such as the parallelism of the GPU, Multiprocessing on multiple CPU cores, more efficient algorithms such as the Barnes Hut and more useful datastructures such as the KDTree. 
- Code by Michael Sherif naguib 7/22/19

### Imports

In [4]:
#imports (may take some time...)
from __future__ import print_function#A requirement of ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual# A python library for GUI interaction
from scipy.spatial import KDTree as KDTree# A library for constructing and querying a KDTree
import ipywidgets as widgets# A python library for GUI interaction
from numba import cuda, jit# An optimization library that can speed up code and even run on GPU
from tkinter import *# Python's library for creating GUI
import math# Python's library for math functions etc,,,
import random# Python's library for random numbers
import tqdm# A progress logger bar library for iteration etc...
import pptk# Static 3d point cloud visualization lib for large 10mil+ datapoints
import time# Python Library for time related things
import open3d# A library for LIDAR point cloud visualization and so much more 1k points
import numpy as np

### Utilities & Etc Needed for Settings

In [5]:
phi = (1 + math.sqrt(5))/2#Prespecify a useful constant: The Golden Ratio
def softmax(inputList,b=1):#Calculate the softmax values for a list returning a new list
    #inputList: a list of numbers to calculate the corresponding softmax values for
    #b=1: a default value saying to use e as the base of exponentiation ... see this wikipedia article for info
    #     https://en.wikipedia.org/wiki/Softmax_function
    assert( not(len(inputList)==0))
    expSum = sum([math.exp(inputList[i]) for i in range(0,len(inputList))])#Sum e^x_i
    #note! could cache the exp calculations for e^x_i ... might save computationally especially if x_i is 'large'
    return [math.exp(inputList[i])/expSum for i in range(0,len(inputList))]# x_i --> e^x_i/(sum)     

### Settings

In [6]:
#================ SETTINGS
#=== Boid Quantity
boidQuantity = int(math.pow(10,2))#                                      Quantity of boids in simulation
#=== Screen Display
height = 400 #                                                           Screen height 
dimensions = 3#                                                          Dimensions ex. 2D x,y 3D x,y,z (inclusive..)
posMax = np.array([height,int(height*phi),int(height*phi)])#             World bounds for Positions x,y,z max 
posMin = np.zeros(3)#                                                    x,y,z min ... use golden ratio for nice view
assert(posMin.size == posMax.size == dimensions)#         (err check)
#=== Kinematics 
maxForceMag = 2*math.pow(math.pi,2)#                                     Maximum magnitude for force update
maxAccelMag = math.pow(math.pi,2)#                                       Maximum magnitude for acceleration update
maxVelMag = math.pi#                                                     Maximum magnitude for velocity update
#=== Boid Properties
mass = np.ones(boidQuantity)#np.random.uniform(low=0.0001,high=2.0,size=boidQuantity)#         sets the mass of the boid
active = np.random.randint(0,high=2,size=boidQuantity,dtype=bool)#            an easy way to dynamically set a boid as active in the simulation
geneCountPer = 5#                                                        specifies allocation for the gene...
genomes = np.random.rand(boidQuantity,geneCountPer)#                     so we can run a genetic algorithm
boidScale = 1#                                                           the size of the dot to draw representing it
#=== Rule Distances
ruleCount = 3
maxDist = (1/2)*(math.sqrt(sum([math.pow(posMax[i]-posMin[i],2) for i in range(dimensions)])))# max dist = world bound max /2 because it wraps
minDist = 1
ruleDistances = np.random.uniform(low=minDist,high=maxDist,size=(boidQuantity,ruleCount))
#=== Screen Settings
backgroundColor = 'white'
boidColor = 'black'
#=== Rule Settings
''' 
    We want the forces to interact in a relative manner ==> i.e one force is stronger than the other to a certain degree
    Therfore we can use the softmax function to convert our forces to a distribution
    Returns a new list containing the results of the softmax function on each... b sets for different bases default e=> b=1
'''
ruleList = []# a list holding the customized rule functions... (see section for initing these...)
distance_scale = 50 # set the influence zone... #order: Cohere,aligm,seperate
ruleDistances= [distance_scale*normDist for normDist in softmax([1,0.75,0.5])]#make the distances relative scaled by some factor
ruleWeights = softmax([0.5,0.2,0.6])#Make the weight relative...

### Initilization & Setup

In [7]:
#=== Init Kinematics Rand
pos = []
#distanceMatrix = np.zeros(boidQuantity,boidQuantity)#                    init a matrix of 0's for positions...  (NOT NEEDED SINCE USING KD TREE...)                  
vel = [np.random.rand(dimensions) for _ in range(boidQuantity)]#        init random vals on 0,1
accel = [np.random.rand(dimensions) for _ in range(boidQuantity)]#      init random vals on 0,1
for i in range(dimensions):#                                             init random positions in bound
    pos.append(np.random.uniform(low=posMin[i],high=posMax[i],size=boidQuantity))
for b in range(boidQuantity):#                                         init random velocities 
    #Convert to Unit Vector and scale according to random withine max
    l = math.sqrt(sum([math.pow(vel[b][i],2) for i in range(dimensions)]))
    r = maxVelMag*random.random()
    for i in range(dimensions):
        vel[b][i] = (vel[b][i]/l)*r       
for b in range(boidQuantity):#                                         init random accelerations 
    #Convert to Unit Vector and scale according to random withine max
    l = math.sqrt(sum([math.pow(accel[b][i],2) for i in range(dimensions)]))
    r = maxVelMag*random.random()
    for i in range(dimensions):
        accel[b][i] = (accel[b][i]/l)*r
for _ in range(0,boidQuantity):#                                         init positions
    pos.append(np.array([random.randint(posMin[i],posMax[i]) for i in range(0,dimensions)]))

### Create objects for: Initial Data, Settings, Configurations

In [8]:
#Make it easier to pass all the data to the simulation class and access it withine that class
initdata = {# intial data as well as storage for the calculation (will be modified during the calculation)
    "position":pos,
    "velocity":vel,
    "acceleration":accel,#technically not needed.... more for observational purposes... if desired
    "mass":mass
}
settings={# the settings for running the simulation
    "boidQuantity":boidQuantity,
    "minPositions":posMin,
    "maxPositions":posMax,
    "maxForceMag":maxForceMag,
    "maxAccelerationMag":maxAccelMag,
    "maxVelocityMag":maxVelMag,
    "boidScale":boidScale,
    "active":active,
    "boidColor":boidColor
}
configurations={# the configured rules for the interactions...
    "ruleList":ruleList,
    "ruleDistances":ruleDistances,
    "ruleWeights":ruleWeights
}

### Screen Setup & Definitions for Functions

In [9]:
#================ Screen Defs
def createGraph(posMax,posMin,backgroundColor):#Create a tkinter canvas to draw to...
    #Creates a tkinter canvas with the specified properties:
    #posMin: a vector (as a list) contiaining the minimum bounds for the screen in the x,y,z ... k for any n dimensional vector
    #posMax: a vector (as a list) contiaining the minimum bounds for the screen in the x,y,z ... k for any n dimensional vector
    #backgroundColor: the background color of the canvas specified by color name string 'white' or by rgb array (255,255,255)
    root = Tk()
    _width,_height = (posMax[0]-posMin[0]),(posMax[1]-posMax[1])
    root.geometry('%dx%d+%d+%d' % (_width, _height, (root.winfo_screenwidth() - _width) / 2, (root.winfo_screenheight() - _height) / 2))
    root.bind_all('<Escape>', lambda event: event.widget.quit())
    graph = Canvas(root, width=_width, height=_height, background=backgroundColor)
    graph.pack()
    return graph
def drawBoids(tkinter_graph,posList,activeList,boidScale,boidColor):#Draw the Boids
    #this function takes a tinketer canvas (graph) and looks at all the positions posList and if the 
    #item is active  (specified by ) activeList 1=active 0=not then it draws the boid as a circle at that
    #position with the scale and color specified.. scale and color is shared among all the boids...
    #(NOTE) depending on how the code for the simulation utilizes the CPU it might be beneficial to 
    #run the graph drawing in a separate thread then syncronize before the next loop ==> i.e let the next calculation
    #run while the previous is being drawn or something in this manner
    #=== Draw the swarm
    graph.delete(ALL)#                  Clear the previous screen
    for p in posList:#for every position ==> O(n)
        if activeList[i]==1:#if the boid is active...
            #draw a circle whose center is at the current position of the boid and with a radius of the scale factor..
            d2Cord = (p[0] + boidScale,#lower left x     the coordinates for the bounding corners of the circle
                      p[1] +boidScale,#lower left y      this keeps the boid in the center
                      p[0] - boidScale,#upper right x
                      p[1] - boidScale)#upper right y
            #draw the boid ... note using syntatic sugar for position update
            tkinter_graph.create_oval(d2Cord,fill=boidColor)
    graph.update()#Update the screen...

### Boid Utilities (Old Code Modified)

In [10]:
class BoidUtils:
    def __init__(self):
        pass
    @staticmethod
    def calcMag(npArray):
        return math.sqrt(sum([math.pow(npArray[i],2) for i in range(npArray.size())]))
    @staticmethod
    def limitMag(npArray,maxMag):
        mag = BoidUtils.calcMag(npArray)
        if mag>maxMag:
            return npArray*(maxMag/mag)
        else:
            return npArray
    #=== Predefined Update rules! these may be used withine the BoidSwarm ... custom rules can be added...
    @staticmethod
    def CohereForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):# ALL RULES IMPLEMENT THIS SAME ARGs SIGNATURE
        '''
        :description: calculates the limited weighted coherernce force for a given boid 
        :param boidIdx: the index of the current boid for which the calculation is being done
        :param dataObject: a dictionary containing: position:[[x,y,z]...] velocity:[[x,y,z]...] ... etc... acceleration mass
        :param calculatedNeighborsIdx: a list of indicies for the position list containing the neighboring positions 
        :param sett:  the setting object reference
        :param conf: the config object reference
        :param ruleIdx: the index of the configuration weights for this rule...
        :return: the force to be applied limited by etc...
        '''        
        # store
        pos_sum = np.zeros(len(dataObject["position"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return pos_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            #Calculate the center position weighted by mass
            pos_sum= np.add(pos_sum,dataObject["position"][nIndex]*dataObject["mass"][nIndex]) 
        #Calculate the target and return the limited weighted force
        target = pos_sum * (1/count)
        desired = BoidUtils.limitMag(np.subtract(target,dataObject["position"][boidIdx]),sett["maxVelocityMag"])
        steer = np.subtract(desired,dataObject["velocity"][boidIdx])
        return conf["ruleWeight"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]
            
    @staticmethod
    def SeparateForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):
        # store
        pos_sum = np.zeros(len(dataObject["position"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return pos_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            #Calculate the center position weighted by mass
            diff = np.subtract(dataObject["position"][boidIndx],dataObject["position"][nIndex])
            edgeDist = BoidUtils.calcMag(diff)# the distance between the points
            pos_sum= np.add(pos_sum,diff*(1/(math.pow(edgeDist,2))))# equivilant to taking the norm of diff then weighing it by 1/edgedist
        #Calculate the target and return the limited weighted force
        target = pos_sum * (1/count)
        steer = np.subtract((sett["maxVelocityMag"]/BoidUtil.calcMag(target))*target,dataObject["velocity"][boidIdx])        
        return conf["ruleWeight"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]    
    
    @staticmethod
    def  AlignForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):
        # store
        vel_sum = np.zeros(len(dataObject["velocity"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return vel_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            vel_sum= np.add(vel_sum,dataObject["velocity"][nIndex]) 
        #Calculate the target and return the limited weighted force
        target = vel_sum * (1/count)
        steer = np.add((sett["maxVelocityMag"]/BoidUtil.calcMag(target))*target,dataObject["velocity"][boidIdx])        
        return conf["ruleWeight"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]
    @staticmethod
    def WallForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):
        # if a boid goes out of bounds for a certain dimension add a force in the opposite direction...  that is
        # polynimially proportional to the distance==> max force seams like a good cap...
        force = np.zeros(len(dataObject["velocity"][0]))
        #for each component of the boids position make a force that increases polynomially with the distance past the bounds...
        for dim in range(force.size()):#for each component
            if dataObject["position"][boidIdx][dim] < sett["minPosition"][dim]: 
                force[dim] += math.pow(sett["minPosition"][dim] - dataObject["position"][boidIdx][dim],2)
            if dataObject["position"][boidIdx][dim] > sett["maxPosition"][dim]: 
                force[dim] += math.pow(dataObject["position"][boidIdx][dim]-sett["maxPosition"][dim],2)
        #NOTE WALLFORCE IS NOT LIMITED ... We create an opposing force that grows with the squared distance in each dim
        return force
        

### Rule Definitions & Config

In [11]:
#================ Setup The Boid Rules...
configurations["ruleList"].append(BoidUtils.CohereForce)
configurations["ruleList"].append(BoidUtils.AlignForce)
configurations["ruleList"].append(BoidUtils.SeparateForce)
assert(len(configurations["ruleList"])==len(configurations["ruleWeights"])==len(configurations["ruleDistances"]))#err check

#Special force for bounding the simulation
configurations["ruleList"].append(BoidUtils.WallForce)#NO WEIGHT or DISTANCE IS USED WITH THIS FORCE

### Boid Simulation Class

In [12]:
class BoidSimulation():
    def __init__(self,dat,sett,conf,recordPositionHistory=False):
        self.dat = dat
        self.sett = sett
        self.conf = conf
        self.rph = recordPositionHistory
        self.ph = []#position history ... updated after each timestep if applicable (assumes no changes to active durring game)

    def time_step(self):#Runs a timestep of the simulation... 
        #=== Calculate the next timestep for the swarm
        #TODO
        # build KD tree O(nlogn)
        # Query tree for each boid(n boids) for each rule (k rules) ==> O(nklogn) < O(n^2)  
        #                                   (note k is a constant and for most k where k is small)
        # Calculate the forces for all of the points returned by the querying criteria O(cnk) (where c = the average # of points returned for each...)
        # Bound the positions,velocities,accelerations O(n)
        tree = KDTree(pos)
        
        #Calculate the neighbors for each ruleset for each boid all in parallel?
        #Calculate the position updates in parallel...?
        for i in range(0,len(self.sett["boidQuanity"])):# n boids
            #Only calculate for the active boids
            if self.sett["active"][i]==1:#if the boid is active... then we want to run the calculation...
                #Calculate the neighbors for a certain rule
                forceSumForBoid= np.zeros(len(self.dat["acceleration"][0]))# just use one of the variables to get the desired dim
                for r in range(0,len(self.conf["ruleList"])):# k rules....
                    #for each rule compute the neighbors ... find the forces update etc..
                    neighborIndxs = tree.query_ball_point(self.dat["position"][i],self.conf["ruleDistances"][r])# O(logn)
                    #Calculate the force update by getting the function for the update... (rules limit force... & soft bound)
                    update = self.conf["ruleList"][r](i,self.dat,neighborIndxs,self.sett,self.conf,r) 
                    #Add the force
                    forceSumForBoid = np.add(forceSumForBoid,update)
                #Now apply the force to the boid ... factoring in the boids mass first
                accelUpdate = forceSumForBoid * (1/self.dat["mass"][i])
                #Update the acceleration velocity and position
                self.dat["acceleration"][i] = accelUpdate
                self.dat["velocity"][i] = BoidUtils.limitMag(np.add(self.dat["velocity"][i],self.dat["acceleration"][i]),self.sett["maxVelocityMag"])
                self.dat["position"][i] = np.add(self.dat["position"][i],self.dat["velocity"][i])               
        #Should we save the data?
        if self.rph:
            self.ph.append(copy.deepcopy(self.dat["position"]))
        #DONE

### Controls 

In [13]:
#=============== Controls Setup
#TODO: slider for each rule distance & weight
#      scatter button
#      3D snapshot ==> pause simulation and use pptk to Draw the 3d frame of the simulation...

#well the last hour was spent trying to get sliders to work ... the issue is that the functions can only recieve the 
#updated value and rerun however the function does not update the value of an outside variable... these are not at 
#all similar to 'event based' despite it being called such... 
# FOR NOW: this feature of the code will be put on halt

### Simulation Loop

In [14]:
#================ Simulation Loop
graph = createGraph(posMax,posMin,backgroundColor)
sim = BoidSimulation(initdata,settings,configurations)
while True:
    #============================================================================
    sim.time_step()
    #============================================================================
    #=== Draw the swarm & Update the screen
    drawBoids(graph,sim.dat["position"],sim.sett["active"],sim.sett["active"],sim.sett["active"],sim.sett["boidScale"],sim.sett["boidColor"])
      

ValueError: not enough values to unpack (expected 2, got 1)

In [1]:
#Here I have been expiramenting with the code of open3d!  this is a very great visualization library... 
#it is perfect for viewing the simulation in 3d ... It is fairly good at handeling loads of a little under 1k points in live time
#and is only limited by the extra computation done between each frame...
#the computation could be done in advance then played back in 3d? ==> HUGE memory requirements but it never needs to be accessed
#until later ... 
#NOTE! jupyter has to be restarted each time this is run because they have a bug in their code that causes GLFW lib to break
#easy fix ...just reload ... remember to save...
'''
import open3d as o3d
import numpy as np
#generate some neat n times 3 matrix using a variant of sync function
x = np.linspace(-3, 3, 401)
mesh_x, mesh_y = np.meshgrid(x, x)
z = np.sinc((np.power(mesh_x, 2) + np.power(mesh_y, 2)))
z_norm = (z - z.min()) / (z.max() - z.min())
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(z_norm, -1)
print('xyz')
print(xyz)

# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.visualization.draw_geometries([pcd])
'''
import open3d
import time
pcd = open3d.PointCloud()

vis = open3d.Visualizer()
vis.create_window()
vis.add_geometry(pcd)


render_option = vis.get_render_option()
render_option.point_size = 5#0.01
#render_option.point_color_option = open3d.visualization.PointColorOption.Normal
import numpy as np
points = np.random.uniform(low=0,high=500,size=(500,3))#500 data points randomized in 3d space
to_reset_view_point = True
while True:
    points = np.mod(points + np.random.uniform(low=-10,high=10,size=(500,3)),500)#500 data points randomized in 3d spacs
    
    pcd.points = open3d.Vector3dVector(points)
    pcd.colors = open3d.Vector3dVector([[0,0,0] for i in range(0,500)])
    vis.update_geometry()
    if to_reset_view_point:
        vis.reset_view_point(True)
        to_reset_view_point = False
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.01)# could be interpreted as frames per second assuming the calculation is instant and retrieval is as well

vis.destroy_window()

KeyboardInterrupt: 

4